In [ ]:
!mkdir msdir
!mkdir input
!mkdir ouput

In [ ]:
import stimela
import drastic

In [ ]:
from ipywidgets import widgets, interact

In [ ]:
def generate_data():
    ms_f = widgets.Text(
        value='',
        placeholder='meerkat_SourceRecovery.ms',
        description='MS file:',
        disabled=False
    )
    ref_freq = widgets.Text(
        value='1.42GHz',
        placeholder='1.42GHz',
        description='Ref Freq (incl units):',
        disabled=False
    )
    noise_psf_weight = widgets.Checkbox(
        value=False,
        description='Run noise-psf vs briggs',
        disabled=False
    )
    gen_data = widgets.Checkbox(
        value=True,
        description='Simulate Data',
        disabled=False
    )
    add_ns = widgets.Checkbox(
        value=True,
        description='Add noise',
        disabled=False
    )
    add_smearing = widgets.Checkbox(
        value=False,
        description='Add smearing',
        disabled=False
    )
    chans_in = widgets.IntSlider(
        value=16,
        min=1,
        max=64,
        step=1,
        description='Channels:',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='d'
    )
    sefd = widgets.IntSlider(
        value=551,
        min=0,
        max=1000,
        step=1,
        description='SEFD (Jy):',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='d'
    )
    tel_name = widgets.Dropdown(
        options=['meerkat', 'vla', 'wsrt', 'kat7'],
        value='meerkat',
        description='Telescope:',
        disabled=False
    )
    synthesis_t = widgets.FloatSlider(
        value=6,
        min=0,
        max=100,
        step=0.5,
        description='Synthesis Time (hr):',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='.1f'
    )
    integration_t = widgets.FloatSlider(
        value=60,
        min=0,
        max=1000,
        step=1,
        description='Integration Time (s):',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='d'
    )
    return (noise_psf_weight,ms_f,tel_name,gen_data,add_ns,add_smearing,
            ref_freq,chans_in,sefd,synthesis_t,integration_t)

In [ ]:
for w in generate_data():
    display(w)